In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [49]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [50]:
import nltk

In [51]:
 nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [52]:
train_df=pd.read_csv('/content/drive/My Drive/Colab_Notebooks/Datasets/hatespeech_kaggle/train.csv')
train_df.head(100)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
95,96,0,@user i'll always hope that one day i'll get t...
96,97,0,#model i love u take with u all the time in ...
97,98,0,couple having sex fat naked japanese girls
98,99,0,#hump on that #hump day #humpersð© @ edwa...


In [53]:
test_df=pd.read_csv('/content/drive/My Drive/Colab_Notebooks/Datasets/hatespeech_kaggle/test.csv')
test_df.head(100)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."
...,...,...
95,32058,great to see you! look forward to welcoming yo...
96,32059,jackblair - na: #horny #hot #naughty #nasty ...
97,32060,@user @user did u take both of them and pour t...
98,32061,ððððâ¤ï¸ððð» happy fathe...


In [54]:
import string

punctuations = string.punctuation

from nltk.corpus import stopwords

stopword = stopwords.words("english")

from nltk.stem.wordnet import WordNetLemmatizer

lem = WordNetLemmatizer()

# creating a function to clean the text
def _clean(text):
    
    #converting to lower case 
    clean_text = text.lower()
    
    # removing punctuations
    clean_text = "". join(p for p in clean_text if p not in punctuations)
    
    # removing stopwords
    words = clean_text.split()
    words =[w for w in words if w not in stopword]
     # lemmatization
    words = [lem.lemmatize(word, 'v') for word in words]
    words = [lem.lemmatize(word, 'n') for word in words]
    
    cleaned_text = " ".join(words)
    
    return cleaned_text

# checking if our function works correctly or not
_clean("this is a Test Text for cleaning")

'test text clean'

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [55]:
train_df["cleaned_tweets"] = train_df["tweet"].apply(_clean)
test_df["cleaned_tweets"] = test_df["tweet"].apply(_clean)
train_df.head()

,id,label,tweet,cleaned_tweets
0,1,0,@user when a father is dysfunctional and is s...,user father dysfunctional selfish drag kid dys...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thank lyft credit cant use cause don...
2,3,0,bihday your majesty,bihday majesty
3,4,0,#model i love u take with u all the time in ...,model love u take u time urð± ðððð...
4,5,0,factsguide: society now #motivation,factsguide society motivation


In [56]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

cvz = CountVectorizer()
word_tfidf = TfidfVectorizer()
cvz.fit(train_df["cleaned_tweets"].values)
count_vector_train = cvz.transform(train_df["cleaned_tweets"].values)
count_vector_test = cvz.transform(test_df["cleaned_tweets"].values)

word_tfidf.fit(train_df["cleaned_tweets"].values)
word_vector_train = word_tfidf.transform(train_df["cleaned_tweets"].values)
word_vector_test = word_tfidf.transform(test_df["cleaned_tweets"].values)

In [57]:
train_vector = count_vector_train
test_vector = count_vector_test
target = train_df['label']

In [58]:
trainx, valx, trainy, valy = train_test_split(train_vector,target)

In [59]:
from sklearn.linear_model import LogisticRegression

In [60]:
model = LogisticRegression()
model.fit(trainx,trainy)
preds = model.predict(valx)
accuracy_score(preds,valy)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.961331497935177

In [61]:
from sklearn.metrics import confusion_matrix,f1_score
confusion_matrix(valy,preds)

array([[7396,   36],
       [ 273,  286]])

In [62]:
f1_score(valy,preds)

0.6492622020431329